In [14]:
from gurobipy import *  
import numpy as np


def Direct_MIP_Sshape_solver(s,timebound):
    # Model
    OP = Model("DMS")  
    I = len(s.Ki)
    J = s.number_of_batch
    numberperunit = int(s.number_of_order / s.number_of_batch)
    K = len(s.Ik)
    B = len(s.mb)
    index=[i for i in range(len(s.mb))]
    d=tupledict(zip(index,s.mb))
    d =tupledict({k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)})
    fd=tupledict(zip(d.keys(),index)) # f(b)
    gd=tupledict(zip(index,d.keys())) # f^-1 (b)
    y = OP.addVars(I, J, vtype=GRB.BINARY, name="y")  #定义决策变量
    z = OP.addVars(J, K, vtype=GRB.BINARY, name="z")
    w = OP.addVars(J, B, lb=0, vtype=GRB.INTEGER, name="w")
    o = OP.addVars(J, B, vtype=GRB.BINARY, name='o')
    u = OP.addVars(J, B, vtype=GRB.INTEGER,name='u')
    delta = OP.addVars(J, B, vtype=GRB.BINARY, name="delta")

    # The objective is to minimize the costs

    OP.setObjective(s.alpha * quicksum(z[j, k] for j in range(J) for k in range(K)) +
        (1 - s.alpha) * (quicksum(2*o[j,b]*s.mb[b] for j in range(J) for b in range(B))+s.number_of_batch*len(s.mb)), GRB.MINIMIZE)
    # The constraints
    OP.addConstrs(sum(y[i, j] for i in range(I)) == numberperunit for j in range(J))
    OP.addConstrs(sum(y[i, j] for j in range(J)) == 1 for i in range(I))
    OP.addConstrs(min(numberperunit, len(s.Ik[k])) * z[j, k] >= sum(y[i, j] for i in s.Ik[k])
                  for j in range(J) for k in range(K))
    OP.addConstrs(min(numberperunit, len(s.Ib[b])) * delta[j, b] >= sum(y[i, j] for i in s.Ib[b])
                  for j in range(J) for b in range(B))

    OP.addConstrs(w[j,b]<= sum(delta[j,b] for b in range(B)) for j in range(J) for b in range(B))
    OP.addConstrs(w[j,b]<= delta[j,b]*(fd[b]+1) for j in range(J) for b in range(B))
    OP.addConstrs(w[j,b]>= delta[j,b] for j in range(J) for b in range(B))
    OP.addConstrs(w[j,gd[k+1]] - w [j,gd[k]] == w[j,gd[k+1]]*(1-delta[j,gd[k]])+delta[j,gd[k]] for j in range(J) for k in range(B-1))
    OP.addConstrs(w[j,b]==2*u[j,b]+o[j,b] for j in range(J) for b in range(B))
    # Set time limit and log file location
    OP.Params.timelimit = timebound
    OP.Params.LogToConsole = 0
    FileStr = './Log/' + 'Direct_MIP_Sshape_solver' + '_' + str(s.number_of_order) + '_' + 'order' + '_' + str(s.number_of_batch) + '_' + 'batch' + '.log'
    OP.Params.LogFile = FileStr

    # Solve

    OP.update()  
    OP.optimize()  
    OP.setParam("SolutionNumber",0)
    vy=OP.getAttr("xn",y)
    matrix=[[] for j in range(J)]
    for j in range(J):
        matrix[j].extend([i for i,x in enumerate(vy.select('*',j)) if x>0.5])
    s.Direct_MIP_Sshape_matrix=matrix
    return OP.ObjVal,OP.MIPGap